In [1]:
import pickle

In [2]:
import matgl

In [3]:
from matgl.ext.ase import PESCalculator, MolecularDynamics, Relaxer

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from disalloy.cellfunc import jitter_ia, jitter_ic, compute_distortion

In [5]:
from disalloy.grid_run import run_packet

In [6]:
pot = matgl.load_model("M3GNet-MP-2021.2.8-PES")

/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_std", torch.tensor(da

In [7]:
relaxer_2021_2_8_PES = Relaxer(pot)

In [8]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)

In [9]:
def relaxer_direct_2021_2_8_PES(structure):
    return relaxer_2021_2_8_PES.relax(structure)
def relaxer_jitter_a_01_2021_2_8_PES(structure, ia = 0.01):
    structure = jitter_ia(structure, ia)
    return relaxer_2021_2_8_PES.relax(structure)#, fmax = 0.001)
def relaxer_jitter_c_005_2021_2_8_PES(structure, ic = 0.01):
    structure = jitter_ic(structure, ic)
    return relaxer_2021_2_8_PES.relax(structure)#, fmax = 0.001)

In [10]:
a = structure_DB.loc[0, 'init_structure']
a_copy = a.copy()
print(a_copy.cell.array,'\n',a_copy.cell.cellpar(),'\n',a_copy.positions)
print('jitter_ia')
jitter_ia(a_copy)
compute_distortion(a, a_copy, quiet = False)
print(a_copy.cell.array,'\n',a_copy.cell.cellpar(),'\n',a_copy.positions)
a_copy = a.copy()
print('jitter_ic')
jitter_ic(a_copy)
compute_distortion(a, a_copy, quiet = False)
print(a_copy.cell.array,'\n',a_copy.cell.cellpar(),'\n',a_copy.positions)

[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]] 
 [ 2.12132034  2.12132034  2.12132034 60.         60.         60.        ] 
 [[0. 0. 0.]]
jitter_ia
Distorsion measure:
0.0000
[[0.79370053 0.79370053 0.        ]
 [0.         0.79370053 0.79370053]
 [0.79370053 0.         0.79370053]] 
 [ 1.12246205  1.12246205  1.12246205 60.         60.         60.        ] 
 [[ 0.00346794 -0.0041085  -0.00693521]]
jitter_ic
Distorsion measure:
0.0060
[[ 0.79606744  0.7946377   0.00438723]
 [-0.00219174  0.79509734  0.7948102 ]
 [ 0.7930717   0.00106721  0.79179825]] 
 [ 1.1248074   1.12423656  1.12067316 60.07932485 59.71671814 59.93291725] 
 [[0. 0. 0.]]


In [12]:
for i in range(8):
    
    a = structure_DB.loc[i,'init_structure'].copy()
    master_copy = a.copy()
    print(master_copy.get_chemical_symbols())
    print(master_copy.cell.cellpar(),'\n',a_copy.positions)

    a_copy = a.copy()
    print('direct')
    relaxer_direct_2021_2_8_PES(a_copy)
    print(a_copy.cell.cellpar(),'\n',a_copy.positions)
    compute_distortion(master_copy, a_copy, quiet = False)
    
    print('jitter_ia')
    a_copy = a.copy()
    relaxer_jitter_a_01_2021_2_8_PES(a_copy)
    print(a_copy.cell.cellpar(),'\n',a_copy.positions)
    compute_distortion(master_copy, a_copy, quiet = False)
    
    print('jitter_ic')
    a_copy = a.copy()
    relaxer_jitter_c_005_2021_2_8_PES(a_copy)
    print(a_copy.cell.cellpar(),'\n',a_copy.positions)
    compute_distortion(master_copy, a_copy, quiet = False)
    break

['Al']
[ 1.12246205  1.12246205  1.12246205 60.         60.         60.        ] 
 [[0. 0. 0.]]
direct
[ 1.08073187  1.08067308  1.08075266 60.00044123 59.99607641 60.00094662] 
 [[0. 0. 0.]]
Distorsion measure:
0.0001
jitter_ia
[ 1.08073195  1.08067317  1.08075271 60.0004468  59.99607528 60.00094199] 
 [[ 0.00182491  0.0054181  -0.00250893]]
Distorsion measure:
0.0001
jitter_ic
[ 1.04503834  1.04683437  1.04285455 60.05850126 60.10039062 59.81200315] 
 [[0. 0. 0.]]
Distorsion measure:
0.0047


In [13]:
df = structure_DB.copy()

In [14]:
relaxer_dic_2021_2_8_PES = {'calculator_label': 'relaxer_dic_2021_2_8_PES',
                            'relaxer': relaxer_direct_2021_2_8_PES}

In [15]:
run_packet(df, relaxer_dic_2021_2_8_PES, 5, 2)

[14, 43, 16, 33, 10]


In [16]:
len(df.attrs[relaxer_dic_2021_2_8_PES['calculator_label']]['calculated'])

5

In [17]:
df[df[relaxer_dic_2021_2_8_PES['calculator_label']].notnull()]

,init_structure,size,relaxer_dic_2021_2_8_PES
10,"(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('C...",2,"((Atom('Al', [6.333481526433646e-07, 8.6804716..."
14,"(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('V...",2,"((Atom('Al', [-1.7882955356262696e-06, -5.8058..."
16,"(Atom('Co', [0.0, 0.0, 0.0], index=0), Atom('C...",2,"((Atom('Co', [-9.717820495221134e-07, -2.31031..."
33,"(Atom('Mn', [0.0, 0.0, 0.0], index=0), Atom('N...",2,"((Atom('Mn', [-3.182622948395819e-06, -1.61884..."
43,"(Atom('Co', [0.0, 0.0, 0.0], index=0), Atom('C...",2,"((Atom('Co', [3.20771218636209e-07, -1.4569891..."


In [18]:
## now we run with another relaxer
relaxer_dic_jitter_c_005_2021_2_8_PES = {
    'calculator_label': 'relaxer_dic_jitter_c_005_2021_2_8_PES',
    'relaxer': relaxer_jitter_c_005_2021_2_8_PES
}

In [19]:
## and we run the same structures for comparison
run_packet(df, relaxer_dic_jitter_c_005_2021_2_8_PES, 50, 2, copy_calculated= 'relaxer_dic_2021_2_8_PES')

[33, 10, 43, 14, 16]


In [20]:
run_packet(df, relaxer_dic_jitter_c_005_2021_2_8_PES, 50, 2, copy_calculated= 'relaxer_dic_2021_2_8_PES')

no more structures to run of this size


In [21]:
calculated_index = df[df[relaxer_dic_2021_2_8_PES['calculator_label']].notnull()].index


In [22]:
for i, j,k  in zip(df.loc[calculated_index, 'init_structure'], 
                df.loc[calculated_index, 'relaxer_dic_2021_2_8_PES'].array, 
             df.loc[calculated_index, 'relaxer_dic_jitter_c_005_2021_2_8_PES'].array):
    print(compute_distortion(i, j[0]))
    print(compute_distortion(i, k[0]))
    print()

0.08171670991394009
0.08166277357433269

0.022529365287150514
0.02322317991696847

0.01929429646574994
0.02078546657726485

0.0067257354750165105
0.008730287298227519

0.04627730708397882
0.040753538829881135



In [114]:
from ase.build import bulk
from ase.calculators.espresso import Espresso, EspressoProfile
from ase.optimize import LBFGS


In [113]:

rocksalt = bulk('NaCl', crystalstructure='rocksalt', a=6.0)

# Pseudopotentials from SSSP Efficiency v1.3.0
pseudopotentials = {'Na': 'na_pbe_v1.5.uspp.F.UPF', 'Cl': 'cl_pbe_v1.4.uspp.F.UPF'}

# Optionally create profile to override paths in ASE configuration:
profile = EspressoProfile(
    command='/path/to/pw.x', pseudo_dir='/path/to/pseudopotentials'
)

calc = Espresso(profile=profile, pseudopotentials=pseudopotentials)

rocksalt.calc = calc

rocksalt.get_potential_energy()  # This will run a single point calculation

opt = LBFGS(rocksalt)

opt.run(fmax=0.005)  # This will run a geometry optimization using ASE's LBFGS algorithm

# Print lattice constant...
print((8 * rocksalt.get_volume() / len(rocksalt)) ** (1.0 / 3.0))